# Interactive visualization

In [ ]:
import os
import pandas as pd
import json
import folium
from IPython.display import display, HTML
from ipywidgets import widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
europe_unemployment_data = pd.DataFrame.from_csv('data/europe_unemployment.tsv', sep='\t', header=0)
europe_unemployment_data.head()

## Description taken from eurostat: 

Unemployment rates represent unemployed persons as a percentage of the labour force. The labour force is the total number of people employed and unemployed. Unemployed persons comprise persons aged 15 to 74 who were: a. without work during the reference week, b. currently available for work, i.e. were available for paid employment or self-employment before the end of the two weeks following the reference week, c. actively seeking work, i.e. had taken specific steps in the four weeks period ending with the reference week to seek paid employment or self-employment or who found a job to start later, i.e. within a period of, at most, three months. This table does not only show unemployment rates but also unemployed in 1000 and as % of the total population. 

In [ ]:
europe_unemployment_data_2016 = europe_unemployment_data[europe_unemployment_data.index.str.match(".*PC_ACT.*")]['2016 '].reset_index()

europe_unemployment_data_2016.columns = ['Country', 'Rate']
europe_unemployment_data_2016['Country'] = europe_unemployment_data_2016['Country'].str.extract('(..$)', expand=False)

In [ ]:
europe_unemployment_data_2016.at[35, 'Country']= 'GB'

In [ ]:
europe_json = json.load(open("topojson/europe.topojson.json"))
m = folium.Map([47.870833, 9.085721], tiles='cartodbpositron', zoom_start=4)
m.choropleth(europe_json, 
             data=europe_unemployment_data_2016, 
             columns=['Country', 'Rate'], 
             key_on='feature.id', 
             fill_color='YlOrRd',
             topojson='objects.europe',
             )
m

# 2 Swiss unemployment data
## 2.a Counting people with jobs, looking for another job

In [ ]:
swiss_unemployment_data = pd.read_csv('data/swiss_unemployment_total.csv')
swiss_unemployment_rates = swiss_unemployment_data.copy()[['Canton','Taux de chômage (1 cav)','Taux de chômage des jeunes (1 pdv)']]
swiss_unemployment_rates.head()

In [ ]:
cantons_dict = {'Zurich': 'ZH', 'Berne': 'BE','Lucerne':'LU','Uri':'UR','Schwyz':'SZ',
    'Obwald':'OW','Nidwald':'NW','Glaris':'GL','Zoug':'ZG','Fribourg':'FR',
    'Soleure':'SO','Bâle-Ville':'BS','Bâle-Campagne':'BL','Schaffhouse':'SH',
    'Appenzell Rhodes-Extérieures':'AR','Appenzell Rhodes-Intérieures':'AI',
    'St-Gall':'SG','Grisons':'GR','Argovie':'AG','Thurgovie':'TG','Tessin':'TI',
    'Vaud':'VD','Valais':'VS','Neuchâtel':'NE','Genève':'GE','Jura':'JU'}

swiss_unemployment_rates.Canton.replace(cantons_dict, inplace=True)

In [ ]:
swiss_json = json.load(open("topojson/ch-cantons.topojson.json"))

map_ = folium.Map([46.819393, 8.333115], tiles='cartodbpositron', zoom_start=8)
map_.choropleth(swiss_json, 
             data=swiss_unemployment_rates, 
             columns=['Canton', 'Taux de chômage (1 cav)'], 
             key_on='feature.id', 
             fill_color='YlOrRd',
             topojson='objects.cantons'
             )
map_

## 2.b Unemployment, without people with job

# 3 Swiss unemployment, split

## 3.a Split by nationality

In [ ]:
swiss_unemployment_nationality_data = pd.read_csv('data/swiss_unemployment_nationality_split.csv')
swiss_unemployment_nationality_rates = swiss_unemployment_nationality_data.copy()[['Canton','Taux de chômage (1 cav)','Nationalité']]

swiss_unemployment_nationality_rates.Canton.replace(cantons_dict, inplace=True)

In [ ]:
swiss_unemployment_foreign_rates = swiss_unemployment_nationality_rates[swiss_unemployment_nationality_rates['Nationalité'] =='Etrangers']
swiss_unemployment_swiss_rates = swiss_unemployment_nationality_rates[swiss_unemployment_nationality_rates['Nationalité'] =='Suisses']

In [ ]:
map_ = folium.Map([46.819393, 8.333115], tiles='cartodbpositron', zoom_start=8)
map_.choropleth(swiss_json, 
     data=swiss_unemployment_swiss_rates, 
     columns=['Canton', 'Taux de chômage (1 cav)'], 
     key_on='feature.id', 
     fill_color='YlOrRd',
     topojson='objects.cantons',
     threshold_scale=[1, 3, 5, 7, 9, 11]
)

map_

In [ ]:
map_ = folium.Map([46.819393, 8.333115], tiles='cartodbpositron', zoom_start=8)
map_.choropleth(swiss_json, 
    data=swiss_unemployment_foreign_rates, 
    columns=['Canton', 'Taux de chômage (1 cav)'], 
    key_on='feature.id', 
    fill_color='YlOrRd',
    topojson='objects.cantons',
    threshold_scale=[1, 3, 5, 7, 9, 11]
)
map_

In [ ]:
def on_click_b(x):
    map_ = folium.Map([46.819393, 8.333115], tiles='cartodbpositron', zoom_start=8)
    if x:
        map_.choropleth(swiss_json, 
             data=swiss_unemployment_swiss_rates, 
             columns=['Canton', 'Taux de chômage (1 cav)'], 
             key_on='feature.id', 
             fill_color='YlOrRd',
             topojson='objects.cantons',
                 threshold_scale=[1, 3, 5, 7, 9, 11]
                )
    else:
        map_.choropleth(swiss_json, 
                 data=swiss_unemployment_foreign_rates, 
                 columns=['Canton', 'Taux de chômage (1 cav)'], 
                 key_on='feature.id', 
                 fill_color='YlOrRd',
                 topojson='objects.cantons',
                 threshold_scale=[1, 3, 5, 7, 9, 11]
                )
    
    display(map_)

w = widgets.Dropdown(options={'Swiss': True, 'Foreigner': False},description='Selection')
interact(on_click_b,x=w)

## 3.b Split by nationality and age

In [ ]:
def to_numeric(x):
    return(x.str.replace('\'', '')).astype('int64')

In [ ]:
swiss_unemployment_nationality_age_data = pd.read_csv('data/swiss_unemployment_nationality_age_split.csv')
swiss_unemployment_nationality_age_rates = swiss_unemployment_nationality_age_data.copy()

swiss_unemployment_nationality_age_rates = swiss_unemployment_nationality_age_rates.drop(['Unnamed: 3','Mois'], 1)
swiss_unemployment_nationality_age_rates.loc[:,"Janvier 2016":"Décembre 2016"] = swiss_unemployment_nationality_age_rates.loc[:,"Janvier 2016":"Décembre 2016"].apply(lambda x: to_numeric(x), axis=1)
swiss_unemployment_nationality_age_rates["Mean"] = (swiss_unemployment_nationality_age_rates.loc[:,"Janvier 2016":"Décembre 2016"].sum(axis=1)/12).astype('int64')

swiss_unemployment_nationality_age_rates = swiss_unemployment_nationality_age_rates[['Canton','Nationalité','Classes d\'âge 15-24, 25-49, 50 ans et plus','Mean']]

swiss_unemployment_nationality_age_rates = swiss_unemployment_nationality_age_rates[swiss_unemployment_nationality_age_rates['Canton'] != 'Total']
swiss_unemployment_nationality_age_rates = swiss_unemployment_nationality_age_rates[swiss_unemployment_nationality_age_rates['Nationalité'] != 'Total']
swiss_unemployment_nationality_age_rates = swiss_unemployment_nationality_age_rates[swiss_unemployment_nationality_age_rates['Classes d\'âge 15-24, 25-49, 50 ans et plus'] != 'Total']

swiss_unemployment_nationality_age_rates['Nationality and age'] = swiss_unemployment_nationality_age_rates['Nationalité'] + swiss_unemployment_nationality_age_rates['Classes d\'âge 15-24, 25-49, 50 ans et plus'] 

swiss_unemployment_nationality_age_rates = swiss_unemployment_nationality_age_rates.pivot(index='Canton', columns= 'Nationality and age', values='Mean')

display(swiss_unemployment_nationality_age_rates)



In [ ]:
swiss_unemployment_nationality_age_rates.plot.barh(grid =False,fontsize= 30,width = 0.85)
rcParams['figure.figsize'] = 40,40
plt.xlabel('Rate',fontsize = 45)
plt.ylabel('Canton', fontsize=45,rotation = 0)
plt.legend(
   fontsize=40)
plt.show()

# 4  Röstigraben Delimitation

In [ ]:
map_ = folium.Map([46.919393, 8.334115], tiles='cartodbpositron', zoom_start=8)

unemployment_swiss_dict = swiss_unemployment_swiss_rates.set_index('Canton')['Taux de chômage (1 cav)']
unemployment_foreign_dict = swiss_unemployment_foreign_rates.set_index('Canton')['Taux de chômage (1 cav)']

foreign_mean = unemployment_foreign_dict.mean()
swiss_mean = unemployment_swiss_dict.mean()

foreign_min = unemployment_foreign_dict.min()
swiss_min = unemployment_swiss_dict.min()

foreign_max = unemployment_foreign_dict.max()
swiss_max = unemployment_swiss_dict.max()


unem_swiss_unem_foreign_color = '#ae017e'
unem_swiss_em_foreign_color = '#fc4e2a'
em_swiss_unem_foreign_color = '#1d91c0'
em_swiss_em_foreign_color = '#fde0dd'



def my_color_function(feature):
    if unemployment_swiss_dict[feature['id']] > swiss_mean:
        if unemployment_foreign_dict[feature['id']] > foreign_mean:
            return unem_swiss_unem_foreign_color
        else:
            return unem_swiss_em_foreign_color
    else:
        if unemployment_foreign_dict[feature['id']] > foreign_mean:
            return em_swiss_unem_foreign_color
        else:
            return em_swiss_em_foreign_color
    

folium.TopoJson(
    swiss_json,
    object_path ='objects.cantons',
    style_function=lambda feature: {
        'fillColor': my_color_function(feature),
    }
).add_to(map_)

step = cm.StepColormap(
    [em_swiss_em_foreign_color, unem_swiss_em_foreign_color],
   vmin= swiss_min,vmax = swiss_max,
    index=[swiss_min, swiss_mean,swiss_max],
    caption='Unemployment rate of Swiss for low unemployment rate of Foreigners'
).add_to(map_)

step = cm.StepColormap(
    [em_swiss_unem_foreign_color, unem_swiss_unem_foreign_color],
    vmin= swiss_min,vmax = swiss_max,
    index=[swiss_min, swiss_mean,swiss_max],
    caption='Unemployment rate of Swiss for high unemployment rate of Foreigners'
).add_to(map_)



print(swiss_min, swiss_mean,swiss_max)
map_